In [ ]:
import os
import json
import pandas as pd
import traceback
#conda install ipykernel -y this code to resolve below error then i have cloed then i reponed it was worked to cnt ipykernel

: 

In [34]:
import os
import json
import pandas as pd
import traceback

In [35]:
from dotenv import load_dotenv
load_dotenv() # it takes values  .env from our local folder whatever the values key

True

In [36]:
# inside folder structure i m gng to create .env file where i kept my api key
KEY=os.getenv("OPENAI_API_KEY") # this line will pick key in our folder structure

In [37]:
#print(KEY) # here we got our key # if get any error load the env package
# When i run this code it was showin my key so i hidden it and recommit

In [38]:
from langchain.chat_models import ChatOpenAI # in class we use langchain.llm and import openAI this lib also do same things

llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo-0613",temperature=0.5)  #gpt-3.5-turbo this wasnt worked for me i changed model then fine
# we are going to paste entire key instead do some

In [39]:
# we gng to implement whtever we learn from past
#lib 

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [40]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    }

}

In [41]:
# we going to give template and question
TEMPLATE='''
Text:{text}
You are an expert MCQ maker.Give then above text ,its is your job to create quiz of
{number} MCQ questions for a {subject} student in {tone} tone.
Make sure questions are not repeated and check all the questions to be confireming the text as well.
make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
###RESPONSE_JSON
{response_json}

'''

quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [42]:
# use LLM chain to connect several components
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)


In [43]:
# Reviewing template
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\

You need to evaluate the complexity of the question and give a complete analysis of the quiz. 
Only use at max 50 words for if the quiz is not at per with the cognitive and analytical abilities of the students,\

if the quiz is not at per with cognitive ana analytical abilities for the student,\

update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities

Quiz MCQs:

{quiz}

Check from an expert English Writer of the above quiz



"""

review_template=PromptTemplate(input_variables=["subject","quize"],template=TEMPLATE2)

In [44]:
review_chain=LLMChain(llm=llm,prompt=review_template,output_key="review",verbose=True)

In [45]:
# Create Obj SequentialChain
#we are gng to pass all the I/P nd O/P vabls
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],
                input_variables=["text","number","subject","tone","response_json"],
                output_variables=["quiz","review"])

In [46]:
#Read data from text file

file_path=r"E:\Python\New folder\AIprojects\MCQGen\data.txt"

with open(file_path,'r') as file:
    TEXT=file.read()
print(TEXT) # I can read my data

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contributed

In [47]:
# Importent 
# Serialize python dictionary into  JSON-FORMATED strin which means our dict result will be come same dict format string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [48]:
NUMBER=5
SUBJECT="machine learning",
TONE="simple"

In [49]:
#getopenai_call_back this will track each and everything (Token Detail)
# langchain offical documentation get_openai_callback()

#tracking our token
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        })
    



> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Othe

In [50]:
# Once its excute we can see no of token which we used for both IP/OP we can see total cost as well in Doller
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Token:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")
#Every thing we have got here

Total Tokens:1727
Prompt Tokens:1363
Completion Token:364
Total Cost:0.0027725000000000002


In [51]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [53]:
# get quiz
quiz=response.get("quiz")

In [55]:
quiz=json.loads(quiz) # we have got same format that we passed as json response

In [59]:
# convert our quiz and ans to DataFrame
quiz_table_data=[]

for key,value in quiz.items():
    # print(key) this is will take how quize like 1,2,3,4 each has quizes that we processed
    mcq=value['mcq']
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items() # we are storing the option and option(mcq)
        ]    
    )
    correct=value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Correct":correct})



In [60]:
pd.DataFrame(quiz_table_data) # we have got the Data Frame of our quize

,MCQ,Choices,Correct
0,What term was coined in 1959 by Arthur Samuel?,a:Artificial intelligence | b:Machine learning...,b
1,Who introduced a theoretical neural structure ...,a:Donald Hebb | b:Arthur Samuel | c:Walter Pit...,a
2,"What was the purpose of Cybertron, developed i...",a:Analyze sonar signals | b:Recognize patterns...,b
3,"According to Tom M. Mitchell, what defines mac...","a:Tasks in cognitive terms | b:Experience, tas...",b
4,What are the two objectives of modern-day mach...,a:Classifying data and predicting future outco...,a


In [61]:
# Convert DF to CSV

quiz=pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv",index=False) #local folder where the file is created

#Done our jupyter implementation